<h1 align=center><font size = 5>Machine Learning models for aiding the decision-making process in emergency departments</font></h1>

<h1>Tabla comparativa de algoritmos<h1>
<h3>Descripción</h3>
<p>
En este script se desarrollará una serie de predicciones basadas en datos del hospital San Juan de Dios Curicó, correspondientes al año 2018 representados por registros de urgencias. El objetivo es predecir, mediante un conjunto de algoritmos, la necesidad de hospitalización de un paciente de urgencias,tomando como input, datos proporcionados por el paciente en la etapa de registro, sus signos vitales registrados en la etapa de triage y el diagnóstico ofrecido por el médico tratante.
Se correrán algoritmos de prodicción tales como árboles y bosques de desición, regresión logística, support vector machines y redes neuronales. Para finalmente evaluar el rendimiento de cada algoritmo en términos de la predicción, mediante indicadores tales como Acuraccy, F1-Score
</p>

<h1 id="Descripción de datos">Descripción de datos</h1>
<p>
Los datos utilizados fueros proporcionados por el Hospital San Juan de Dios, Curicó, Chile y corresponden a 4.971 registros de pacientes que asistieron a urgencias durante el periodo comprendido entre el 1 de enero de 2018 y agosto de 2019, los datos fueron limpiados y transformados en un script desarrollado previamente
<ul>
    <li>Datos: <a href="https://drive.google.com/open?id=1QmJTDCx060GJw3QHyjl5OhmR4szE8Ojh" target="_blank">https://drive.google.com/open?id=1QmJTDCx060GJw3QHyjl5OhmR4szE8Ojh</a></li>
    <li>Tipo de datos: csv</li>
   </ul>
<p>
Las variables presentes en los datos se describen a continuación:
<ul>    
   
   <li><b>PAC_EDAD</b>: corresponde a la edad del paciente en enteros</li>
   <li><b>MOTIVO_CONSULTA</b>: corresponde a la razón por la que el paciente acude al servicio de urgencias string</li>
   <li><b>MEDIO</b>: corresponde al medio de llegada, mediante el que el paciente acude al servicio de urgencias</li>
   <li><b>SEXO</b>: corresponde al sexo del paciente</li>
   <li><b>CAT</b>: corresponde a la categoría de gravedad asignada al paciente en el proceso de Triage</li>
   <li><b>PRESION_SIST</b>: corresponde la presión sistólica del paciente </li>
   <li><b>PRESION_DIAST</b>: corresponde la presióndiastólica del paciente</li>
   <li><b>SATO2</b>: Dato numérico que representa la saturometria del paciente (Nivel de oxigeno en la sangre)</li>
   <li><b>TEMPERATURA</b>: corresponde a la temperatura corporal del paciente en el momento de la categorización</li>
   <li><b>GLASGOW</b>: corresponde a al nivel registrado por el paciente en la escala Glasgow</li>
   <li><b>DM</b>: corresponde si el paciente presenta o no Diabetes Mellitus</li>
   <li><b>EVA</b>: corresponde si se aplica al paciente una evaluación de vias aéreas</li>
   <li><b>HGT</b>: corresponde a la medida de azucar en la sangre del paciente</li>
   <li><b>LCFA</b>: corresponde a si el paciente presenta obstrucción crónica de vías aéreas</li>
   <li><b>FR</b>: corresponde a la frecuencia respiratoria del paciente</li>
   <li><b>HTA</b>: corresponde a si el paciente posee Hipertención Arterial</li>
   <li><b>HORA_INSC</b>: corresponde a la hora en la que el paciente fue categorizado</li>
   <li><b>MIN_INSC</b>: corresponde al minuto en que el paciente fue categorizado</li>
   <li><b>TIEMPO_ESPERA_CAT</b>: corresponde al tiempo que espera el paciente luego de ser registrado, para ser categorizado</li>
      
</ul>
</p>

Cargar paquetes necesarios

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
import sklearn as sk  
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
#Métodos de tuneo de parámetros
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Lectura de datos

In [ ]:
archivo= 'df_ final_media.csv'
df_T_hosp= pd.read_csv(archivo,encoding='latin-1',sep=",")
df_T_hosp.head(10)

Seleccionar las variables relevantes para la predicción y asignarselas a la matriz X, de variables independientes

In [ ]:
df_T_hosp=df_T_hosp[['ID_PACIENTE', 'PAC_EDAD', 'SATO2', 'TEMPERATURA',
       'GLASGOW', 'EVA', 'HGT', 'FR', 'DIAS_HOSP', 'SEXO_M', 'SEXO_F', 'DM_D',
       'DM_N', 'DM_S', 'LCFA_D', 'LCFA_N', 'LCFA_S', 'LCFA_D.1', 'LCFA_N.1']]

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")
dias = df_T_hosp['DIAS_HOSP']
ax = sns.boxplot(x=dias)

In [ ]:
y=df_T_hosp['DIAS_HOSP']

In [ ]:
df_T_hosp.drop(columns =['DIAS_HOSP'], inplace = True)

In [ ]:
X = df_T_hosp

In [ ]:
missing_data = df_T_hosp.isnull()

    
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

 <h1 id="Normalización de datos">Normalización de datos</h1>
<p>
Para aplicar el paquete de arboles de desición, los datos deben estar en una escala similar, es por ello que optamos por normalizarlos para que los valores estén en un rango entre -2 y 2. Esta medida no altera los resultados aunque si faborece a la eficiencia de los algoritmos
</p>

In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X)

 <h1 id="Configurando algoritmos">Configurando algoritmos</h1>
<p>
En esta sección se definen parámetros necesarios para la correcta aplicación de los algoritmos a implementar, además de seccionar el conjunto de datos en datos de prueba(30%)y de entrenamiento (70%). Los parámetros escogidos pueden ser modificados con el fin de obtener resultados diferentes
</p>

Carga de paquetes

In [ ]:
from sklearn.model_selection import train_test_split

Segmentación del conjunto de datos

In [ ]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=5)

Árbol de desición

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
DT= DecisionTreeRegressor(random_state=0)
DT.fit(X_trainset,y_trainset)
yhat_1 = DT.predict(X_testset)
DT_CV = cross_val_score(DT, X, y, cv=10)
DT_R2=round(DT.score(X_testset, y_testset),4)
DT_MAX=max_error( y_testset,yhat_1)
DT_MAD=mean_absolute_error( y_testset,yhat_1)
DT_MSE =round(mean_squared_error(y_testset,yhat_1),4)
DT_r22= round(r2_score(y_testset,yhat_1),4)
DT_MAPE=round(mean_absolute_percentage_error(y_testset,yhat_1),4)
DT_MAPE

In [ ]:
DT_GS = DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=6,
                      max_features=None, max_leaf_nodes=None,
                      min_samples_leaf=100, min_samples_split=2,
                      min_weight_fraction_leaf=0.0,
                      random_state=42, splitter='best')
DT_GS.fit(X_trainset,y_trainset)

In [ ]:
DT_parameters = [{'criterion': ['mse', 'friedman_mse', 'mae'], 'max_depth': np.arange(1, 21)},{'min_samples_leaf': [1, 5, 10, 20, 50, 100]}]
DT_GS = GridSearchCV(DecisionTreeRegressor(random_state=42), DT_parameters, verbose=1, cv=3)
DT_GS.fit(X_trainset,y_trainset)

In [ ]:
DT_GS.best_estimator_

In [ ]:
yhat_1_GS= DT_GS.predict(X_testset)
DT_R2_GS=round(DT_GS.score(X_testset, y_testset),4)
DT_MAX_GS=max_error( y_testset,yhat_1_GS)
DT_MAD_GS=mean_absolute_error( y_testset,yhat_1_GS)
DT_MSE_GS =round(mean_squared_error(y_testset,yhat_1_GS),4)
DT_r22_GS= round(r2_score(y_testset,yhat_1_GS),4)
DT_MAPE_GS=round(mean_absolute_percentage_error(y_testset,yhat_1_GS),4)
DT_MAPE_GS

In [ ]:
resultados_DT = {'índices de rendimiento':['R^2','MAX error','MAD','MSE','MAPE'],
             'Árboles de decisión':[DT_R2,DT_MAX,DT_MAD,DT_MSE,DT_MAPE],
             'Grid Search':[DT_R2_GS,DT_MAX_GS,DT_MAD_GS,DT_MSE_GS,DT_MAPE_GS]}
Tabla_resultados_DT=pd.DataFrame(resultados_DT)
print(Tabla_resultados_DT)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
X = df_T_hosp

In [ ]:
perm = PermutationImportance(DT_GS, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names =X.columns.tolist())

In [ ]:
import shap
ex = shap.TreeExplainer(DT_GS)
shap_values = ex.shap_values(X)
shap.summary_plot(shap_values, X,max_display=9)

Bosque de desición

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF= RandomForestRegressor(max_depth=2, random_state=0)
RF.fit(X_trainset,y_trainset)
yhat_2 = RF.predict(X_testset)
RF_R2=round(RF.score(X_testset, y_testset),4)
RF_MAX=round(max_error( y_testset,yhat_2),4)
RF_MAD=round(mean_absolute_error( y_testset,yhat_2),4)
RF_MSE =round(mean_squared_error(y_testset,yhat_2),4)
RF_MAPE=round(mean_absolute_percentage_error(y_testset,yhat_2),4)
RF_MAPE

In [ ]:
RF_parameters ={ 'n_estimators': [200, 500],'max_features': ['auto', 'sqrt', 'log2'],'max_depth' : [2,3,4,5,6,7,8],'criterion' :['mse', 'friedman_mse', 'mae']}
RF_GS = GridSearchCV(estimator=RF,param_grid=RF_parameters, cv= 3)
RF_GS.fit(X_trainset,y_trainset)

In [ ]:
# Kerner Explainer
explainer = shap.KernelExplainer(RF.predict_proba,X[:100])
shap_values = explainer.shap_values(X[:100])
shap.summary_plot(shap_values, X[:100])

In [ ]:
RF_GS.best_estimator_

In [ ]:
RF_GS = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=8, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)
RF_GS.fit(X_trainset,y_trainset)

In [ ]:
yhat_2_GS= RF_GS.predict(X_testset)
RF_R2_GS=round(RF.score(X_testset, y_testset),4)
RF_MAX_GS=round(max_error( y_testset,yhat_2_GS),4)
RF_MAD_GS=round(mean_absolute_error( y_testset,yhat_2_GS),4)
RF_MSE_GS =round(mean_squared_error(y_testset,yhat_2_GS),4)
RF_MAPE_GS=round(mean_absolute_percentage_error(y_testset,yhat_2_GS),4)
RF_MAPE_GS

In [ ]:
resultados_RF = {'índices de rendimiento':['R^2','MAX error','MAD','MSE','MAPE'],
             'Bosques de decisión':[RF_R2,RF_MAX,RF_MAD,RF_MSE,RF_MAPE],
             'Grid Search':[RF_R2_GS,RF_MAX_GS,RF_MAD_GS,RF_MSE_GS,RF_MAPE_GS]}
Tabla_resultados_RF=pd.DataFrame(resultados_RF)
print(Tabla_resultados_RF)

In [ ]:
import shap
ex = shap.TreeExplainer(RF)
shap_values = ex.shap_values(X)
shap.summary_plot(shap_values, X)

Regresión logística (multi-class)

In [ ]:
from sklearn.linear_model import SGDRegressor
SGDR =SGDRegressor(max_iter=1000, tol=1e-3)
SGDR.fit(X_trainset,y_trainset)
yhat_3= SGDR.predict(X_testset)
SGDR_R2=round(SGDR.score(X_testset, y_testset),4)
SGDR_MAX=round(max_error( y_testset,yhat_3),4)
SGDR_MAD=round(mean_absolute_error( y_testset,yhat_3),4)
SGDR_MSE =round(mean_squared_error(y_testset,yhat_3),4)
SGDR_MAPE=round(mean_absolute_percentage_error(y_testset,yhat_3),4)

In [ ]:
SGDR_GS = SGDRegressor(alpha=0.03, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='elasticnet', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
SGDR_GS.fit(X_trainset,y_trainset)

In [ ]:
SGDR_GS.best_estimator_

In [ ]:
yhat_3_GS= SGDR_GS.predict(X_testset)
SGDR_R2_GS=round(SGDR.score(X_testset, y_testset),4)
SGDR_MAX_GS=round(max_error( y_testset,yhat_3_GS),4)
SGDR_MAD_GS=round(mean_absolute_error( y_testset,yhat_3_GS),4)
SGDR_MSE_GS =round(mean_squared_error(y_testset,yhat_3_GS),4)
SGDR_MAPE_GS=round(mean_absolute_percentage_error(y_testset,yhat_3_GS),4)

In [ ]:
resultados_SGDR = {'índices de rendimiento':['R^2','MAX error','MAD','MSE','MAPE'],
             'SGDR':[SGDR_R2,SGDR_MAX,SGDR_MAD,SGDR_MSE,SGDR_MAPE],
             'Grid Search':[SGDR_R2_GS,SGDR_MAX_GS,SGDR_MAD_GS,SGDR_MSE_GS,SGDR_MAPE_GS]}
Tabla_resultados_SGDR=pd.DataFrame(resultados_SGDR)
print(Tabla_resultados_SGDR)

In [ ]:
# Kerner Explainer
explainer = shap.KernelExplainer(SGDR.predict_proba,X[:100])
shap_values = explainer.shap_values(X[:100])
shap.summary_plot(shap_values, X[:100])

Regresión Lasso

In [ ]:
from sklearn import linear_model
LssR = linear_model.Lasso(alpha=0.1).fit(X_trainset,y_trainset)
yhat_6= LssR.predict(X_testset)
LssR_R2=round(LssR.score(X_testset, y_testset),4)
LssR_MAX=round(max_error( y_testset,yhat_6),4)
LssR_MAD=round(mean_absolute_error( y_testset,yhat_6),4)
LssR_MSE =round(mean_squared_error(y_testset,yhat_6),4)
LssR_MAPE=round(mean_absolute_percentage_error(y_testset,yhat_6),4)

In [ ]:
LssR_parameters ={'alpha':[0.005, 0.02, 0.03, 0.05, 0.06]}
LssR_GS = GridSearchCV(estimator=LssR,param_grid=LssR_parameters, cv= 3, verbose=True, n_jobs=-1)
LssR_GS.fit(X_trainset,y_trainset)

In [ ]:
LssR_GS = LssR(alpha=0.003, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
LssR_GS.fit(X_trainset,y_trainset)

In [ ]:
LssR_GS.best_estimator_

In [ ]:
yhat_6_GS= LssR_GS.predict(X_testset)
LssR_R2_GS=round(LssR.score(X_testset, y_testset),4)
LssR_MAX_GS=round(max_error( y_testset,yhat_6_GS),4)
LssR_MAD_GS=round(mean_absolute_error( y_testset,yhat_6_GS),4)
LssR_MSE_GS =round(mean_squared_error(y_testset,yhat_6_GS),4)
LssR_MAPE_GS=round(mean_absolute_percentage_error(y_testset,yhat_6_GS),4)

In [ ]:
resultados_LssR = {'índices de rendimiento':['R^2','MAX error','MAD','MSE','MAPE'],
             'Regresión Lasso':[LssR_R2,LssR_MAX,LssR_MAD,LssR_MSE,LssR_MAPE],
             'Grid Search':[LssR_R2_GS,LssR_MAX_GS,LssR_MAD_GS,LssR_MSE_GS,LssR_MAPE_GS]}
Tabla_resultados_LssR=pd.DataFrame(resultados_LssR)
print(Tabla_resultados_LssR)

In [ ]:
# Kerner Explainer
explainer = shap.KernelExplainer(LssR.predict_proba,X[:100])
shap_values = explainer.shap_values(X[:100])
shap.summary_plot(shap_values, X[:100])

Redes Neuronales

In [ ]:
from sklearn.neural_network import MLPRegressor
NN = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1).fit(X_trainset,y_trainset)
yhat_4 = NN.predict(X_testset)
NN_R2=round(NN.score(X_testset, y_testset),4)
NN_MAX=round(max_error( y_testset,yhat_4),4)
NN_MAD=round(mean_absolute_error( y_testset,yhat_4),4)
NN_MSE =round(mean_squared_error(y_testset,yhat_4),4)
NN_MAPE=round(mean_absolute_percentage_error(y_testset,yhat_4),4)

In [ ]:
NN_parameters ={'solver': ['lbfgs'], 'max_iter': [500,1000,1500], 'alpha': 10.0 ** -np.arange(1, 7), 'hidden_layer_sizes':np.arange(5, 12), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
NN_GS = GridSearchCV(estimator=NN,param_grid=NN_parameters, cv= 3, verbose=True, n_jobs=-1)
NN_GS.fit(X_trainset,y_trainset)

In [ ]:
NN_GS.best_estimator_

In [ ]:
NN_GS = MLPRegressor(alpha=0.001, hidden_layer_sizes=5, max_iter=500, random_state=5,
             solver='lbfgs')
NN_GS.fit(X_trainset,y_trainset)

In [ ]:
yhat_4_GS= NN_GS.predict(X_testset)
NN_R2_GS=round(NN.score(X_testset, y_testset),4)
NN_MAX_GS=round(max_error( y_testset,yhat_4_GS),4)
NN_MAD_GS=round(mean_absolute_error( y_testset,yhat_4_GS),4)
NN_MSE_GS =round(mean_squared_error(y_testset,yhat_4_GS),4)
NN_MAPE_GS=round(mean_absolute_percentage_error(y_testset,yhat_4_GS),4)

In [ ]:
resultados_NN = {'índices de rendimiento':['R^2','MAX error','MAD','MSE','NN_MAPE'],
             'Red Neuronal':[NN_R2,NN_MAX,NN_MAD,NN_MSE,NN_MAPE],
             'Grid Search':[NN_R2_GS,NN_MAX_GS,NN_MAD_GS,NN_MSE_GS,NN_MAPE_GS]}
Tabla_resultados_NN=pd.DataFrame(resultados_NN)
print(Tabla_resultados_NN)

In [ ]:
data_DF=pd.DataFrame(X, columns=df_T_hosp.columns)

In [ ]:
# Kerner Explainer
import shap
explainer = shap.KernelExplainer(NN.predict,data_DF[:100])
shap_values = explainer.shap_values(data_DF[:100])
shap.summary_plot(shap_values, data_DF[:100])

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
perm = PermutationImportance(NN_GS, random_state=1).fit(data_DF, y)
eli5.show_weights(perm, feature_names =data_DF.columns.tolist())

Support Vector Machine

In [ ]:
from sklearn.svm import LinearSVR
SVM = LinearSVR(random_state=0, tol=1e-5)
SVM.fit(X_trainset, y_trainset) 
yhat_5 = SVM.predict(X_testset)
SVM_R2=round(SVM.score(X_testset, y_testset),4)
SVM_MAX=round(max_error( y_testset,yhat_5),4)
SVM_MAD=round(mean_absolute_error( y_testset,yhat_5),4)
SVM_MSE =round(mean_squared_error(y_testset,yhat_5),4)
SVM_MAPE=round(mean_absolute_percentage_error(y_testset,yhat_5),4)

In [ ]:
SVM_parameters ={'C': [0.1,1, 10, 100]}
SVM_GS = GridSearchCV(estimator=SVM,param_grid=SVM_parameters, cv= 3, verbose=True, n_jobs=-1)
SVM_GS.fit(X_trainset,y_trainset)

In [ ]:
SVM_GS.best_estimator_

In [ ]:
yhat_5_GS= SVM_GS.predict(X_testset)
SVM_R2_GS=round(SVM.score(X_testset, y_testset),4)
SVM_MAX_GS=round(max_error( y_testset,yhat_5_GS),4)
SVM_MAD_GS=round(mean_absolute_error( y_testset,yhat_5_GS),4)
SVM_MSE_GS =round(mean_squared_error(y_testset,yhat_5_GS),4)
SVM_MAPE_GS=round(mean_absolute_percentage_error(y_testset,yhat_5_GS),4)

In [ ]:
resultados_SVM = {'índices de rendimiento':['R^2','MAX error','MAD','MSE'],
             'SVM':[SVM_R2,SVM_MAX,SVM_MAD,SVM_MSE],
             'Grid Search':[SVM_R2_GS,SVM_MAX_GS,SVM_MAD_GS,SVM_MSE_GS]}
Tabla_resultados_SVM=pd.DataFrame(resultados_SVM)
print(Tabla_resultados_SVM)

In [ ]:
resultados = {'Algoritmos de regresión':['Árboles de decisión','Bosques de decisión','Regresión Lasso','Red Neuronal','Support Vector Machine'],
             'R^2':[DT_R2_GS,RF_R2_GS,LssR_R2_GS,NN_R2_GS,SVM_R2_GS],
             'Max error':[DT_MAX_GS,RF_MAX_GS,LssR_MAX_GS,NN_MAX_GS,SVM_MAX_GS],
             'MAD':[DT_MAD_GS,RF_MAD_GS,LssR_MAD_GS,NN_MAD_GS,SVM_MAD_GS],
             'MSE':[DT_MSE_GS,RF_MSE_GS,LssR_MSE_GS,NN_MSE_GS,SVM_MSE_GS],
             'MAPE':[DT_MAPE_GS,RF_MAPE_GS,LssR_MAPE_GS,NN_MAPE_GS,SVM_MAPE_GS]}
Tabla_resultados=pd.DataFrame(resultados)
print(Tabla_resultados)